# Slope Analysis

This project use the change of fluorecent intensity slope to identify responders from calcium imaging experiment.

## Analysis Steps

The `getBaselineAndMaxStimulationSlopeFromCSV` function smoothes the raw data by the moving window decided by `filterSize`, and analyzes the smoothed Ca intensity in an CSV and returns baseline slope and drug slope.

The _slope of baseline_ is calculated as the linear regression slope during the 3 minutes period before stimulation onset.

In addition, the smoothed data are separated into segments which n = regressionSize data points are included. The linear regression slope is then calculated for each segment. 

The _peak slope of stimulation_ is the most negative slope during the chosen stimulation period.

## Set-Up the Environment

In [4]:
%load_ext autoreload
import sys
sys.path.append("../src")
import os
import glob
import slopeTools
import plotTools
import statsTools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats

# Pandas DataFrame as a Database

This script creates a single dataframe containing AFU data for every structure in all experiments. 

Columns define group, drug, distance, structure, etc.

This large dataframe can be queried as needed, or exported as a CSV.

In [5]:
database = pd.DataFrame(columns = ['Group', 'Distance', 'MCN','Sniffer','AFU'])

def addGroup(folderPath, groupName):    
    filePaths = glob.glob(folderPath+"/*.xls")
    filePaths = [x for x in filePaths if x.endswith("um.xls")]
    for filePath in filePaths:
        fileName = os.path.basename(filePath)
        #distance = int(fileName.replace("um.xls", ""))
        addExperiment(filePath, groupName)
        
def addExperiment(filePath, groupName):    
    global database   #global reads a variable outside the function
    df = pd.read_csv(filePath, delimiter="\t")
    roiNames = df.columns[1:] #return to the column labels 
    for roiName in roiNames:
        mcn, distance, sniffer = roiName.split(".")
        distance = int(distance.replace("um", ""))
        afu = df[roiName].values
        row = {'Group': groupName, 'Distance': distance, 'MCN': mcn, 'Sniffer':sniffer, 'AFU':afu}
        database = database.append(row,ignore_index = True)
        
addGroup(R"X:\Data\OT-Cre\OT-GCaMP-nonspecific\04-03-19 evoke OT\04-30-2020 Cs-GLU analyze", "CsGlu")
addGroup(R"X:\Data\OT-Cre\OT-GCaMP-nonspecific\04-03-19 evoke OT\04-30-2020 K-GLU analyze", "KGlu")
addGroup(R"X:\Data\OT-Cre\OT-GCaMP-nonspecific\04-03-19 evoke OT\04-30-2020 L368 - Cs analyze", "L368")

database

,Group,Distance,MCN,Sniffer,AFU
0,CsGlu,25,19514000,C1,"[1286.65983, 1266.2571699999999, 1189.49916999..."
1,CsGlu,25,19514000,C2,"[2948.64467, 2962.93833, 3239.71167, 3417.8625..."
2,CsGlu,25,19514000,C3,"[5736.4195, 5736.50717, 5026.41233, 4580.00867..."
3,CsGlu,25,19514000,C4,"[2283.93833, 2211.74517, 2253.62167, 2301.0938..."
4,CsGlu,25,19514000,C5,"[2045.98417, 1792.38467, 2021.4029999999998, 1..."
...,...,...,...,...,...
441,L368,125,20217028,C15,"[1024.454, 1141.7939999999999, 1079.1860000000..."
442,L368,125,20217028,C16,"[1149.15, 1137.655, 1136.93, 1116.005, 1118.42..."
443,L368,125,20217028,C17,"[3113.2129999999997, 2974.022, 3031.2940000000..."
444,L368,125,20217028,C18,"[1648.366, 1611.486, 1646.211, 1597.274, 1576...."


# Convert raw AFU into dF/F (%)

In [11]:
baselineStartIndex = 20
treatmentStartIndex = 30
experimentPeriod = 5/60 #min
dFoFs = []
analyzed = database.copy()

analyzed["BaselineAFUMeans"] = [np.mean(x[baselineStartIndex:treatmentStartIndex]) for x in analyzed["AFU"].values]
for i in range(len(analyzed["AFU"])):
    dFoF = (analyzed["AFU"][i]-analyzed["BaselineAFUMeans"][i])*100/analyzed["BaselineAFUMeans"][i]
    dFoFs.append(dFoF)
    
analyzed["Raw dF/F (%)"] = dFoFs

analyzed = analyzed.drop(columns=["AFU", "BaselineAFUMeans"], axis=1)


analyzed

,Group,Distance,MCN,Sniffer,Raw dF/F (%)
0,CsGlu,25,19514000,C1,"[26.876577163943114, 24.864685904510754, 17.29..."
1,CsGlu,25,19514000,C2,"[-1.6574346372227533, -1.1807155509507214, 8.0..."
2,CsGlu,25,19514000,C3,"[28.56409879575307, 28.56606364761605, 12.6514..."
3,CsGlu,25,19514000,C4,"[27.972329497133295, 23.927243542925748, 26.27..."
4,CsGlu,25,19514000,C5,"[23.747368607671493, 8.408896655945105, 22.260..."
...,...,...,...,...,...
441,L368,125,20217028,C15,"[-0.5552749665760618, 10.83503053803611, 4.757..."
442,L368,125,20217028,C16,"[11.070408867461687, 9.959366488371503, 9.8892..."
443,L368,125,20217028,C17,"[37.94349143568214, 31.77607130849393, 34.3137..."
444,L368,125,20217028,C18,"[12.440712290178862, 9.925000688955723, 12.293..."


# Smooth raw data by filtersize

In [54]:
filterSize = 5
regressionSize = 10

length = [len(x) for x in analyzed["dF/F (%)"].values]
maxLength = max(length)
time = np.arange(maxLength)*experimentPeriod 
smoothTimes = statsTools.smoothY(time, filterSize)
analyzed["dF/F (%)"] = [statsTools.smoothY(x, filterSize) for x in analyzed["Raw dF/F (%)"].values]
analyzed


,Group,Distance,MCN,Sniffer,Raw dF/F (%),dF/F (%),BaselineMeans
0,CsGlu,25,19514000,C1,"[26.876577163943114, 24.864685904510754, 17.29...","[21.84072307926667, 19.003907158001294, 16.241...","[-2.3868963031317647, -2.01268516260621, -0.42..."
1,CsGlu,25,19514000,C2,"[-1.6574346372227533, -1.1807155509507214, 8.0...","[7.763973194601914, 9.775688043374481, 11.1001...","[-11.234850283022643, -8.666851716381895, -2.4..."
2,CsGlu,25,19514000,C3,"[28.56409879575307, 28.56606364761605, 12.6514...","[9.291335093843292, 10.769249062288777, 13.264...","[-1.864495946558872, 4.521180457844151, 12.141..."
3,CsGlu,25,19514000,C4,"[27.972329497133295, 23.927243542925748, 26.27...","[28.11335145335973, 28.311119053030833, 28.082...","[-9.965659483642105, -8.099140504467874, -2.50..."
4,CsGlu,25,19514000,C5,"[23.747368607671493, 8.408896655945105, 22.260...","[19.80381459331928, 18.16510188118031, 18.1830...","[-8.368519540836678, -4.429513751250882, -0.76..."
...,...,...,...,...,...,...,...
441,L368,125,20217028,C15,"[-0.5552749665760618, 10.83503053803611, 4.757...","[10.466257991487444, 13.406909232091417, 14.97...","[0.24161283516131293, 4.099924320250132, 5.527..."
442,L368,125,20217028,C16,"[11.070408867461687, 9.959366488371503, 9.8892...","[9.279000026418869, 8.943125970974036, 8.57640...","[0.6509029776644514, 1.3871679035208582, 1.667..."
443,L368,125,20217028,C17,"[37.94349143568214, 31.77607130849393, 34.3137...","[27.990417371697415, 25.14774112198889, 22.706...","[-0.6738416006033553, 0.2736999180046426, -0.6..."
444,L368,125,20217028,C18,"[12.440712290178862, 9.925000688955723, 12.293...","[10.585477304434109, 9.142399281266824, 8.3357...","[0.4281666565255944, 0.8705194344178179, 0.767..."


In [30]:
print(max(time))
print(smoothTimes[18])
print(time[1])

print(smoothTimes[1])/

37.083333333333336
1.7083333333333333
0.08333333333333333
0.2916666666666667


# Calculate the slope during baseline and treatment

In [69]:
distances = [25]#, 50, 75, 100, 125]
groups = ["CsGlu"]#, "KGlu", "L368"]

analyzed["BaselineMeans"] = [x[baselineStartIndex:treatmentStartIndex] for x in analyzed["dF/F (%)"].values]
baselineStartIndex = 20 + 2 # index is right shifted for 2.5 indexes after smoothing
treatmentStartIndex = 30 + 2 


for group in groups:
    responseByDistance = []
    for distance in distances:
        baselineSlopes =[]
        stimulationSlopes =[]
        rows = analyzed.loc[(database['Group'] == group) & (database['Distance'] == distance)]
        
        
        baselineDFoFs = [x[baselineStartIndex:treatmentStartIndex] for x in rows['dF/F (%)'].values]        
        baselineTime = time[baselineStartIndex:treatmentStartIndex]

        for baselineDFoF in baselineDFoFs:
            baselineSlope, baselineIntercept, r, p, stdErr = scipy.stats.linregress(baselineTime, baselineDFoF)
            baselineSlopes.append(baselineSlope)

        smoothedIntensities = [x for x in rows['dF/F (%)'].values]
        for smoothedIntensity in smoothedIntensities:
            segments = statsTools.getMovingWindowSegments(smoothedIntensity, regressionSize) 
            segSlopes = slopeTools.getAllSegmentSlopes(segments, experimentPeriod)   
            segTimesOffset = (regressionSize * experimentPeriod)
            segTimes = statsTools.smoothY(smoothTimes, filterSize)
            treatmentStartTime = smoothTimes[treatmentStartIndex]
            treatmentEndTime = treatmentStartTime + 2.5
            treatmentSlopeMax = statsTools.rangeMax(segSlopes, segTimes, treatmentStartTime, treatmentEndTime)
            treatmentSlopeMaxIndex = segSlopes.index(treatmentSlopeMax)
            treatmentSlopeMaxTime = segTimes[treatmentSlopeMaxIndex]
            stimulationSlopes.append(treatmentSlopeMax)
            print(f"stimulation slope: {treatmentSlopeMax} %/min")        

stimulation slope: 12.710697361243035 %/min
stimulation slope: 24.096666148776 %/min
stimulation slope: 35.68624511878947 %/min
stimulation slope: 15.119956355545256 %/min
stimulation slope: 17.51048496823199 %/min
stimulation slope: 14.320018585426231 %/min
stimulation slope: 24.26125525437619 %/min
stimulation slope: 3.4864795069459498 %/min
stimulation slope: 16.57831397542909 %/min
stimulation slope: -0.7919932448721788 %/min
stimulation slope: -0.9595921376987102 %/min
stimulation slope: -1.1084155811942946 %/min
stimulation slope: 0.3536971227732299 %/min
stimulation slope: -2.170517077779816 %/min
stimulation slope: -2.0183538157477474 %/min
stimulation slope: -0.877540207837634 %/min
stimulation slope: -0.801043193069258 %/min
stimulation slope: -1.291228359378459 %/min
stimulation slope: 4.8637489788833115 %/min
stimulation slope: 8.736780873612618 %/min
stimulation slope: 7.326169187069734 %/min
stimulation slope: 11.517951016617598 %/min
stimulation slope: 5.771417166131379 

## Define CSV Files and Filter Settings

The user can list the ABF files they want to analyze

In [ ]:
def getBaselineAndMaxStimulationSlopeFromCSV(rawTimes,rawIntensity, filterSize = 15, regressionSize = 15):
    """
    This method analyzes fluorecent intensuty in a CSV and returns baseline slope and stimulation slope.
    
    Arguments:
        filterSize: number of points (sweeps) for the moving window average
        regressionSize: number of points (sweeps) to use to calculate regression slopes during the stimulation range
        
    Returns:
        baseline regression slope (over full range)
        peak stimulus regression slope (regression over defined size)
    """
    experimentPeriod = 5/60 #min
    ######
    ##########
    #########

    # determine the time range of stimulus and baseline
    stimulationTimeStart = rawTimes[30]
    stimulationSearchWidth = 30 # index
    stimulationTimeEnd = rawTimes[30 + stimulationSearchWidth]
    baselineTimeStart = stimulationTimeStart-1.5
    baselineTimeEnd = stimulationTimeStart
    
    plt.axvspan(stimulationTimeStart, stimulationTimeEnd, color='r', alpha=.1)#############

    # determine baseline region based on stimulation time
    baselineTimeStart = stimulationTimeStart-1.5
    baselineTimeEnd = stimulationTimeStart
    
    baselineMean = np.mean(rawIntensity[20:30])
    rawDFoF = (rawIntensity - baselineMean)*100/baselineMean
    plt.plot(rawTimes, rawDFoF, '.', alpha=.5)
    smoothIntensity, smoothTimes = statsTools.smoothY(rawDFoF, rawTimes, filterSize)
    plt.plot(smoothTimes, smoothIntensity, '-')
    
    
    baselineIndexStart, baselineIndexEnd = statsTools.rangeIndex(smoothTimes, baselineTimeStart, baselineTimeEnd)
    baselineIntensity = smoothIntensity[baselineIndexStart:baselineIndexEnd]
    plt.axvspan(baselineTimeStart, baselineTimeEnd, color='b', alpha=.1)

    # isolate smoothed baseline intensity
    baselineIntensity = smoothIntensity[baselineIndexStart:baselineIndexEnd]
    baselineTimes = smoothTimes[baselineIndexStart:baselineIndexEnd]
    baselineSlope, baselineIntercept, r, p, stdErr = scipy.stats.linregress(baselineTimes, baselineIntensity)
    # calculate linear regression of baseline region
    baselineRegressionXs = np.linspace(baselineTimeStart, baselineTimeEnd)
    baselineRegressionYs = baselineRegressionXs * baselineSlope + baselineIntercept
    plt.plot(baselineRegressionXs, baselineRegressionYs, color='b', ls='--')
    print(f"Baseline slope: {baselineSlope} AFU/min")

    # perform a moving window linear regression on the smoothed Intensity
    segments = statsTools.getMovingWindowSegments(smoothIntensity, regressionSize)
    segSlopes = slopeTools.getAllSegmentSlopes(segments, experimentPeriod)   
    segTimesOffset = (regressionSize * experimentPeriod)
    segTimes = np.arange(len(segSlopes)) * experimentPeriod + segTimesOffset    
    plt.subplot(212, sharex = ax1)
    plt.plot(segTimes, segSlopes, '.')

    # search the stimulation range for the most positive slope
    plt.axvspan(stimulationTimeStart, stimulationTimeEnd, color='r', alpha=.1)
    stimulationSlopeMax = statsTools.rangeMax(segSlopes, segTimes, stimulationTimeStart, stimulationTimeEnd)
    stimulationSlopeMaxIndex = segSlopes.index(stimulationSlopeMax)
    stimulationSlopeMaxTime = segTimes[stimulationSlopeMaxIndex]
    print(f"stimulation slope: {stimulationSlopeMax} AFU/min")
    plt.axvline(stimulationSlopeMaxTime, color='r', ls='--')
    plt.axhline(stimulationSlopeMax, color='r', ls='--')
    plt.axhline(baselineSlope, color='b', ls='--')

    plt.ylabel("Slope (AFU/min)")
    plt.xlabel("Time (minutes)")

    plt.show()
    
    return baselineSlope, stimulationSlopeMax

In [ ]:
csvFilePaths = [R"X:\Data\OT-Cre\OT-GCaMP-nonspecific\04-03-19 evoke OT\04-30-2020 L368 - Cs analyze\075um.xls",
                #R"X:\Data\OT-Cre\OT-GCaMP-nonspecific\04-03-19 evoke OT\04-30-2020 Cs-glu analyze\025um.xls"
               ]

The users can decide the parameters they want for data analysis. 

`filterSize` decides number of points (sweeps) for the moving window average. 

`regressionSize` decides the number of smoothed data points used to calculate linear regression slopes during the stimulation range.

In [ ]:
filterSize = 5
regressionSize = 10

## Analyze All CSVs

In [ ]:
baselineSlopes = []
stimulationSlopes = []
for csvFilePath in csvFilePaths:
    csvData = pd.read_csv(csvFilePath, delimiter="\t")
    roiNames = csvData.columns[1:]
    framePeriodSeconds = 5.0
    framesPerMinute = 60 / framePeriodSeconds
    times = csvData['Time'].values / framesPerMinute


    for roiName in roiNames:
        plt.figure(figsize=(8, 6))
        ax1 = plt.subplot(211)
        plt.title(roiName)
        plt.ylabel("dF/F (%)")
        rawIntensity = csvData[roiName].values
        baselineSlope, stimulationSlope = getBaselineAndMaxStimulationSlopeFromCSV(times, rawIntensity, filterSize, regressionSize)

        baselineSlopes.append(baselineSlope)
        stimulationSlopes.append(stimulationSlope)

## Compare Baseline vs. Stimulation Slopes

The users can plot the basleine slope and the peak stimulation slope of each cell, and report the p-value in the title by performing a paired t-test between baseline slopes and peak stimulation slopes.

In [ ]:
plotTools.plotPairs(baselineSlopes, stimulationSlopes, "slopes")

## Assess Responsiveness of All Cells

Generate a scatter plot showing the slope difference of each cell.

In [ ]:
slopeThreshold = 15
stimulationEffects = []
for i in range(len(roiNames)):
    stimulationEffects.append(stimulationSlopes[i] - baselineSlopes[i])

plt.figure (figsize=(12, 4))
plt.ylabel("Slope Difference (%/min)")
plt.plot(roiNames, stimulationEffects, 'o', color = "b")
plt.gca().set_xticklabels(roiNames, rotation=45, ha='right')
plt.axhline(slopeThreshold, color='r', ls='--')
plt.show()

## Define Cells as Responsive vs. Non-Responsive

The users can define the <b>slopeThreshold</b>. The difference between baseline slope and peak stimulation slope must be more negative than this value to be a responder
slopeThreshold

In [ ]:
stimulationEffects=statsTools.responderByDelta(roiNames, stimulationEffects, slopeThreshold)